In [1]:
import csv
import random
import numpy as np
import tensorflow as tf
import pandas as pd
from datasets import Dataset
from transformers import DataCollatorForTokenClassification
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

2024-07-22 09:23:43.638974: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-22 09:23:43.640600: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-22 09:23:43.645623: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-22 09:23:43.658692: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 09:23:43.680220: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
import sys
sys.setrecursionlimit(3000)


In [1]:
import torch

In [4]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [5]:
from huggingface_hub import notebook_login
notebook_login()


In [6]:
model_name_or_path = "/root/autodl-tmp/model/modelscope/Llama-2-7b-ms"
tokenizer_name_or_path = "/root/autodl-tmp/model/modelscope/Llama-2-7b-ms"

# Model

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer

/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


LlamaTokenizerFast(name_or_path='TinyLlama/TinyLlama-1.1B-Chat-v1.0', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path,

                                                           num_labels=2,
                                                           ignore_mismatched_sizes=True,
                                                           device_map="auto",
                                                          force_download=True)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Dataset

In [10]:
file_path = "/root/autodl-tmp/Data_enhanced/enhanced_Cross_dataset.csv"

In [11]:
def read_csv_as_list(file_path):
  data = []
  with open(file_path, 'r', newline='', encoding='utf-8', errors='ignore') as file:
    csv_reader = csv.reader(file)
    first_row = True
    for row in csv_reader:
      if first_row:  
        first_row = False
        continue
      data.append(row)
  return data
csv_data = read_csv_as_list(file_path)

In [12]:
random.shuffle(csv_data)
dataset_array = np.array(csv_data)
df = pd.DataFrame(dataset_array, columns=["high_requirement", "low_requirement", "label"])
arrow_dataset = Dataset.from_pandas(df)

In [13]:
tokenizer.padding_side = "right" 

In [14]:
def process_function(example,tokenizer=tokenizer):
    MAX_LENGTH = 256    
    input_ids, attention_mask, labels = [], [], []
    "high_requirement", "low_requirement"
    instruction = tokenizer("\n".join(["high_requirement: " + example["high_requirement"] +
                                       "low_requirement: " + example["low_requirement"]]),
                            max_length=MAX_LENGTH, 
                            padding="max_length", 
                            truncation=True,
                            # return_tensors="pt"
                            )
    input_ids = instruction["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = instruction["attention_mask"] + [tokenizer.eos_token_id]
    
    labels = [int(label) for label in example["label"]]
    # labels = example["label"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [16]:
tokenized_ds = arrow_dataset.map(process_function, 
                                remove_columns=arrow_dataset.column_names)

Map:   0%|          | 0/2647 [00:00<?, ? examples/s]

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [17]:
print(tokenized_ds[0]["input_ids"])

[1, 1880, 29918, 12277, 358, 29901, 450, 1788, 4091, 2758, 4824, 8910, 25222, 304, 671, 5264, 6993, 3694, 470, 16083, 2407, 3694, 304, 12439, 22069, 677, 29918, 12277, 358, 29901, 8153, 4495, 29883, 4091, 2094, 2547, 5987, 322, 4386, 599, 3370, 10643, 746, 9068, 399, 1718, 29907, 3757, 4339, 29889, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [18]:
print(tokenized_ds[0]["labels"])

[0]


# Peft

## LoraConfig

In [21]:
from peft import LoraConfig, TaskType, get_peft_model
LORA_R = 32  
LORA_ALPHA = 32
LORA_DROPOUT = 0.3

config = LoraConfig(task_type=TaskType.SEQ_CLS,
                    r = LORA_R, # the dimension of the low-rank matrices
                    lora_alpha = LORA_ALPHA, # scaling factor for the weight matrices
                    lora_dropout = LORA_DROPOUT,
                    bias="none",
                    # target_modules=["query_key_value"],
                    )
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.SEQ_CLS: 'SEQ_CLS'>, inference_mode=False, r=32, target_modules=None, lora_alpha=32, lora_dropout=0.3, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

## P-tuning

In [22]:
# from peft import PromptEncoderConfig, TaskType, get_peft_model, PromptEncoderReparameterizationType
# config = PromptEncoderConfig(
#     task_type = TaskType.SEQ_CLS,
#     num_virtual_tokens=64,
#     encoder_reparameterization_type=PromptEncoderReparameterizationType.MLP,
#     encoder_dropout=0.1,
#     encoder_num_layers=12,
#     encoder_hidden_size=128
# )

## PromptTuning

In [23]:
# from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType
# config = PromptTuningConfig(
#     task_type = TaskType.SEQ_CLS,
#     # prompt_tuning_init = PromptTuningInit.TEXT,
#     num_virtual_tokens=12,
#     prompt_tuning_init_text=
#     '''Consider the fol57.14lowing scenario where \"high_requirement\" and \"low_requirement text\" 
#     represent high-level and low-level requirements of the software system,  respectively. 
#     Your task is to ascertain the traceability between these two requirements:
#     ''',
#     # tokenizer_name_or_path=tokenizer,
# )

In [24]:
model = get_peft_model(model, config)

In [25]:
model.print_trainable_parameters()

trainable params: 4,509,696 || all params: 1,039,026,176 || trainable%: 0.4340


In [26]:
print(model)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.3, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=2048, out_featur

In [27]:
from torch.utils.data import DataLoader

In [28]:
# from transformers import DataCollatorForTokenClassification
# dl = DataLoader(tokenized_ds, 
#                 batch_size=2, 
#                 collate_fn=DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True))

In [29]:
type(tokenized_ds)

datasets.arrow_dataset.Dataset

In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)

    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    }

In [32]:
model.enable_input_require_grads()

In [33]:
train_args = TrainingArguments(output_dir="./cross_model",     
                per_device_train_batch_size=1,  
                per_device_eval_batch_size=1, 
                logging_steps=50,                
                evaluation_strategy="epoch",     
                save_strategy="epoch",           
                save_total_limit=10,             
                learning_rate=2e-5,              
                weight_decay=0.01,              
                metric_for_best_model="f1_score",      
                num_train_epochs = 5,
                load_best_model_at_end=True)     


In [34]:
datasets = tokenized_ds.train_test_split(test_size=0.1)

In [35]:
datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2382
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 265
    })
})

In [36]:
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=datasets["train"],
    eval_dataset=datasets["test"],
    compute_metrics=eval_metric
)

In [37]:
len(datasets["train"]['input_ids'][0])

256

In [ ]:
history = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Score
1,0.511500,0.551710,0.883019,0.892752,0.883019,0.884239


Checkpoint destination directory ./cross_model/checkpoint-2382 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/root/miniconda3/lib/python3.10/site-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /TinyLlama/TinyLlama-1.1B-Chat-v1.0/resolve/main/config.json (Caused by ProxyError('Cannot connect to proxy.', TimeoutError('_ssl.c:980: The handshake operation timed out')))"), '(Request ID: eb18f8ef-3056-41c3-93bb-3ccdb96973b9)') - silently ignoring the lookup for the file config.json in TinyLlama/TinyLlama-1.1B-Chat-v1.0.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in TinyLlama/TinyLlama-1.1B-Chat-v1.0 - will assume that the vocabulary was not modified.
  warnings.warn(


# Result

In [ ]:
trainer.evaluate(datasets["test"])

In [ ]:
file_path

In [ ]:
config

In [ ]:
model